In [2]:
import pandas as pd
import numpy as np 
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Step 2: Sort the DataFrame and reate a dictionary to hold each country's data
csv_file_path = 'dataset\\csv\\international_top_terms_cleaned.csv'
df = pd.read_csv(csv_file_path)
df.head()

<>:7: SyntaxWarning: invalid escape sequence '\c'
<>:7: SyntaxWarning: invalid escape sequence '\c'
C:\Users\roche\AppData\Local\Temp\ipykernel_19544\1119450337.py:7: SyntaxWarning: invalid escape sequence '\c'
  csv_file_path = 'dataset\csv\international_top_terms_cleaned.csv'


,country_code,region_name,region_code,week,score,rank,country_name,term,refresh_date
0,AR,Río Negro,AR-R,2022-11-27,100.0,25,Argentina,Mac Allister,2024-06-21
1,AR,Chubut Province,AR-U,2022-12-18,58.0,25,Argentina,Mac Allister,2024-06-21
2,AR,Cordoba,AR-X,2022-12-18,77.0,25,Argentina,Mac Allister,2024-06-21
3,AR,Jujuy,AR-Y,2022-11-27,98.0,25,Argentina,Mac Allister,2024-06-21
4,AR,Jujuy,AR-Y,2022-12-18,100.0,25,Argentina,Mac Allister,2024-06-21


In [3]:
df.columns

Index(['country_code', 'region_name', 'region_code', 'week', 'score', 'rank',
       'country_name', 'term', 'refresh_date'],
      dtype='object')

In [4]:
basket_Australia = (df[df['country_name'] == "Australia"]
          .groupby(['region_name', 'term'])['score'] 
          .mean().unstack().reset_index().fillna(0)
          .set_index('region_name'))

print(basket_Australia)

term                          Alex de Minaur  Alice Springs  Ash Barty  \
region_name                                                              
Australian Capital Territory       62.000000      57.000000        0.0   
New South Wales                    56.500000       0.000000       73.0   
Northern Territory                100.000000      77.589744      100.0   
Queensland                         75.333333      54.400000        0.0   
South Australia                    72.000000      64.214286        0.0   
Tasmania                           75.000000      61.000000        0.0   
Victoria                           59.000000      65.666667       62.0   
Western Australia                  82.000000      66.750000        0.0   

term                          Billings  Bird flu  Boxing  Cameron Brink  \
region_name                                                               
Australian Capital Territory     100.0       0.0     0.0            0.0   
New South Wales                   

In [5]:
def hot_encode(x): 
    if(x <= 0): 
        return 0
    if(x >= 1): 
        return 1

basket_encoded = basket_Australia.map(hot_encode) 
basket_Australia = basket_encoded

print(basket_Australia)

term                          Alex de Minaur  Alice Springs  Ash Barty  \
region_name                                                              
Australian Capital Territory               1              1          0   
New South Wales                            1              0          1   
Northern Territory                         1              1          1   
Queensland                                 1              1          0   
South Australia                            1              1          0   
Tasmania                                   1              1          0   
Victoria                                   1              1          1   
Western Australia                          1              1          0   

term                          Billings  Bird flu  Boxing  Cameron Brink  \
region_name                                                               
Australian Capital Territory         1         0       0              0   
New South Wales                   

In [6]:
# Building the model 
frq_items = apriori(basket_Australia.astype('bool'), min_support = 0.8, use_colnames = True)

print("After frequent Items")
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 0.5
                          ) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
display(rules)

After frequent Items


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
70,(RBA),(Nvidia),0.875,0.875,0.875,1.000,1.142857,0.109375,inf,1.0
71,(Nvidia),(RBA),0.875,0.875,0.875,1.000,1.142857,0.109375,inf,1.0
236,"(Alex de Minaur, RBA)",(Nvidia),0.875,0.875,0.875,1.000,1.142857,0.109375,inf,1.0
237,"(Alex de Minaur, Nvidia)",(RBA),0.875,0.875,0.875,1.000,1.142857,0.109375,inf,1.0
240,(RBA),"(Alex de Minaur, Nvidia)",0.875,0.875,0.875,1.000,1.142857,0.109375,inf,1.0
...,...,...,...,...,...,...,...,...,...,...
8705,(Full moon),"(Nvidia, Premier League, RBA, Cricket, UFC, Al...",1.000,0.875,0.875,0.875,1.000000,0.000000,1.0,0.0
8706,(Cricket),"(Nvidia, Premier League, RBA, Full moon, UFC, ...",1.000,0.875,0.875,0.875,1.000000,0.000000,1.0,0.0
8707,(UFC),"(Nvidia, Premier League, RBA, Full moon, Crick...",1.000,0.875,0.875,0.875,1.000000,0.000000,1.0,0.0
8708,(Alex de Minaur),"(Nvidia, Premier League, RBA, Full moon, Crick...",1.000,0.875,0.875,0.875,1.000000,0.000000,1.0,0.0
